In [ ]:
!pip install mljar-supervised --user

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import numpy as np 
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns

import re
import warnings
from supervised.automl import AutoML
import xgboost as xgb

warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('../input/water-potability/water_potability.csv')

In [ ]:
df.head()

In [ ]:
total = df.isnull().sum().sort_values(ascending=False)
percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total,percent],axis=1,keys=['Total','Percent'])
missing_data

In [ ]:
df['ph']=df['ph'].fillna(df.groupby(['Potability'])['ph'].transform('mean'))
df['Sulfate']=df['Sulfate'].fillna(df.groupby(['Potability'])['Sulfate'].transform('mean'))
df['Trihalomethanes']=df['Trihalomethanes'].fillna(df.groupby(['Potability'])['Trihalomethanes'].transform('mean'))

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
corrmat = df.corr()
f, ax = plt.subplots(figsize=(5,5))
sns.heatmap(corrmat,vmax=.2,square = True);

In [ ]:
fig = df[[col for col in df.columns]].hist(figsize=(20, 10))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df[df.columns[:-1]], df["Potability"], test_size=0.20, random_state=2021)

automl = AutoML(eval_metric="accuracy",
                algorithms=['Baseline', 'Linear', 'Decision Tree', 'Random Forest', 'Xgboost', 'Neural Network', 'CatBoost', 'LightGBM'],
                )
automl.fit(X_train, y_train)

predictions = automl.predict(X_test)

In [ ]:
predictions = pd.DataFrame(predictions)
predictions.to_csv('submission.csv', index=False)

In [ ]:
accuracy_score(y_test, predictions)